In [4]:
from keckODL.target import Target, TargetList
from keckODL.offset import Stare, StarSkyStar, SkyStar
from keckODL.block import ObservingBlockList, ScienceBlock, StandardStarBlock
from keckODL.alignment import BlindAlign, GuiderAlign

from astropy import units as u

In [6]:
# Define some targets
feige110 = Target('Feige110')
ngc1333 = Target('NGC1333', rotmode='PA', PA=22.5)

The following test cases are meant to be test particles for ODL implementations.  These are meant not to demonstrate a typical observation (though some do), but to challenge a language to describe something which may be somewhat non-standard.

### 1) A KCWI+KCRM Observation

First, observe a single target with two blue exposures on target and a third on an offset sky position between them (in a star-sky-star pattern).  The offset pattern should use a sky position offset from the target by +60 arcseconds in RA and +30 arcseconds in Dec.  The blue exposures should be 1800s long and simultaneously to the blue exposures, the red side should take a pair of shorter exposures (approximately 900s long each).

Second, observe a standard star observation in a single staring observation with both red and blue exposures of 10 seconds.  This will be a staring observation with no repeats.  The science observation above should store the information that this standard star observation is to be used by the DRP for that science observation.

In [5]:
from keckODL.kcwi import KCWIblueDetectorConfig, KCWIredDetectorConfig, KCWIConfig, mira

In [7]:
# Define my long science exposures
kcwib_1800s = KCWIblueDetectorConfig(exptime=1800, nexp=1, binning='2x2', readoutmode=0, ampmode=9)
kcwir_1800s = KCWIredDetectorConfig(exptime=900, nexp=2, binning='2x2', readoutmode=0, ampmode=9)
kcwir_1800s.match_time(kcwib_1800s.estimate_clock_time())
# Define the short exposures for the standard star
kcwib_10s = KCWIblueDetectorConfig(exptime=10, nexp=1, binning='2x2', readoutmode=0, ampmode=9)
kcwir_10s = KCWIredDetectorConfig(exptime=10, nexp=1, binning='2x2', readoutmode=0, ampmode=9)
# Define my instrument config
med_slicer_b4800_r6500 = KCWIConfig(slicer='medium', bluegrating='BH3', bluecwave=4800,
                                    redgrating='RH3', redcwave=6563)
# Define the offset pattern to use for the science exposures
skyoffset = StarSkyStar(dx=60*u.arcsec, dy=30*u.arcsec, repeat=3)
# Define the standard star OB
std1 = StandardStarBlock(target=feige110, pattern=Stare(),
                         instconfig=med_slicer_b4800_r6500,
                         detconfig=(kcwib_10s, kcwir_10s),
                         align=BlindAlign(),
                        )
# Define the science OB
sci1 = ScienceBlock(target=ngc1333, pattern=skyoffset,
                    instconfig=med_slicer_b4800_r6500,
                    detconfig=(kcwib_1800s, kcwir_1800s),
                    align=BlindAlign(),
                    associated=std1,
                   )

In [11]:
# Show the detector configs
kcwib_1800s.to_dict()['DetectorConfigs'][0]

{'name': 'KCWI 1800s x1',
 'instrument': 'KCWI',
 'detector': 'blue',
 'exptime': 1800,
 'nexp': 1,
 'readoutmode': 0,
 'ampmode': 9,
 'dark': False,
 'binning': '2x2',
 'window': None}

In [12]:
kcwir_1800s.to_dict()['DetectorConfigs'][0]

{'name': 'KCWI 926s x2',
 'instrument': 'KCWI',
 'detector': 'red',
 'exptime': 926.5,
 'nexp': 2,
 'readoutmode': 0,
 'ampmode': 9,
 'dark': False,
 'binning': '2x2',
 'window': None}

In [14]:
kcwib_10s.to_dict()['DetectorConfigs'][0]

{'name': 'KCWI 10s x1',
 'instrument': 'KCWI',
 'detector': 'blue',
 'exptime': 10,
 'nexp': 1,
 'readoutmode': 0,
 'ampmode': 9,
 'dark': False,
 'binning': '2x2',
 'window': None}

In [13]:
kcwir_10s.to_dict()['DetectorConfigs'][0]

{'name': 'KCWI 10s x1',
 'instrument': 'KCWI',
 'detector': 'red',
 'exptime': 10,
 'nexp': 1,
 'readoutmode': 0,
 'ampmode': 9,
 'dark': False,
 'binning': '2x2',
 'window': None}

In [15]:
# Show the instrument config
med_slicer_b4800_r6500.to_dict()['InstrumentConfigs'][0]

{'name': 'medium BH3 4800 A',
 'instrument': 'KCWI',
 'slicer': 'medium',
 'bluegrating': 'BH3',
 'bluefilter': 'KBlue',
 'bluenandsmask': False,
 'bluefocus': None,
 'bluecwave': 4800,
 'bluepwave': None,
 'redgrating': 'RH3',
 'redfilter': 'KRed',
 'rednandsmask': False,
 'redfocus': None,
 'redcwave': 6563,
 'redpwave': None,
 'calmirror': 'Sky',
 'calobj': 'Dark',
 'polarizer': 'Sky',
 'arclamp': None,
 'domeflatlamp': None}

In [17]:
# Show the custom offset pattern
skyoffset.to_dict()['OffsetPatterns'][0]

{'name': 'StarSkyStar (60 30) x3',
 'repeat': 3,
 'offsets': [{'dx': 0.0,
   'dy': 0.0,
   'dr': 0.0,
   'frame': 'SkyFrame',
   'relative': False,
   'posname': 'star',
   'guide': True},
  {'dx': 60.0,
   'dy': 30.0,
   'dr': 0.0,
   'frame': 'SkyFrame',
   'relative': False,
   'posname': 'sky',
   'guide': False},
  {'dx': 0.0,
   'dy': 0.0,
   'dr': 0.0,
   'frame': 'SkyFrame',
   'relative': False,
   'posname': 'star',
   'guide': True}]}

```
# Mock up of the OB description (actual implementation TBD based on whether we want to store references, or the original data)
{'target': ,
 'pattern': ,
 'instconfig': ,
 'detconfig': [],
 'align': ,
 'blocktype': ,
 'associatedblocks': [],
 'guidestar': ,
 'drp_args': {}, # a dict containing arguments for the data reduction pipeline
 'ql_args' {}, # a dict containing arguments for the quick look pipeline
}
```

### 2) A NIRES SPEC+SCAM Observation

Acquire a faint target (which needs sky subtraction in SCAM) on to the NIRES slit and observe it with 5 repeats of an ABBA sequence with one 300 second SPEC exposure and ten 10 second SCAM exposures at each point in the ABBA sequence.

In [20]:
from keckODL.nires import NIRESConfig, NIRESSpecDetectorConfig, NIRESScamDetectorConfig, ABBA, mira

In [22]:
# Define the SPEC and SCAM Exposures
spec_300s = NIRESSpecDetectorConfig(exptime=300, readoutmode='MCDS32', coadds=1)
scam_10s = NIRESScamDetectorConfig(exptime=10, readoutmode='CDS', coadds=1, nexp=6)
# Define a NIRES instrument config
nires = NIRESConfig()
# Define the ABBA pattern to use
abba5 = ABBA(offset=10*u.arcsec, repeat=5)
# Define the OB
OB = ScienceBlock(target=ngc1333, pattern=abba5, instconfig=nires, detconfig=[spec_300s, scam_10s], align=GuiderAlign(bright=False))

In [23]:
# Show the detector configs
spec_300s.to_dict()['DetectorConfigs'][0]

{'name': 'NIRES Spec 300s (MCDS32, 1 coadds) x1',
 'instrument': 'NIRES Spec',
 'detector': '',
 'exptime': 300,
 'nexp': 1,
 'readoutmode': 'MCDS32',
 'coadds': 1}

In [24]:
# Show the detector configs
scam_10s.to_dict()['DetectorConfigs'][0]

{'name': 'NIRES SCAM 10s (CDS, 1 coadds) x6',
 'instrument': 'NIRES SCAM',
 'detector': '',
 'exptime': 10,
 'nexp': 6,
 'readoutmode': 'CDS',
 'coadds': 1}

In [25]:
# Show the instrument config
nires.to_dict()['InstrumentConfigs'][0]

{'name': 'NIRES Instrument Config', 'instrument': 'NIRES'}

In [26]:
# Show the offset pattern
abba5.to_dict()['OffsetPatterns'][0]

{'name': 'ABBA (10.00 arcsec) x5',
 'repeat': 5,
 'offsets': [{'dx': 0.0,
   'dy': 10.0,
   'dr': 0.0,
   'frame': 'NIRES Slit',
   'relative': False,
   'posname': 'A',
   'guide': True},
  {'dx': 0.0,
   'dy': -10.0,
   'dr': 0.0,
   'frame': 'NIRES Slit',
   'relative': False,
   'posname': 'B',
   'guide': True},
  {'dx': 0.0,
   'dy': -10.0,
   'dr': 0.0,
   'frame': 'NIRES Slit',
   'relative': False,
   'posname': 'B',
   'guide': True},
  {'dx': 0.0,
   'dy': 10.0,
   'dr': 0.0,
   'frame': 'NIRES Slit',
   'relative': False,
   'posname': 'A',
   'guide': True}]}

```
# Mock up of the OB description (actual implementation TBD based on whether we want to store references, or the original data)
{'target': ,
 'pattern': ,
 'instconfig': ,
 'detconfig': [],
 'align': ,
 'blocktype': ,
 'associatedblocks': [],
 'guidestar': ,
 'drp_args': {}, # a dict containing arguments for the data reduction pipeline
 'ql_args' {}, # a dict containing arguments for the quick look pipeline
}
```

### 3) A MOSFIRE MOS Mask

Observe a MOS mask using MOSFIRE in spectroscopy mode in the Y, J, H, and K filters.  Use the reccommended exposure time of either 120 or 180 seconds (depending on filter) with MCDS16 readout mode.  For the filters with 180 second exposures, use 5 repeats of an ABBA sequence, for the filters with 120 second exposures use 8 repeats.  The ABBA sequence should have a dither magnitude of 1.25 arcseconds.

### 4) A MOSFIRE Single Object

Observe a single object using MOSFIRE in spectroscopy mode in the Y, J, H, and K filters.  Use the reccommended exposure time of either 120 or 180 seconds (depending on filter) with MCDS16 readout mode.  Use the longslit for Y and J filters with an ABBA pattern and 2 repeats, but use long2pos mask and a long2pos pattern for the H and K observations.

### 5) An Unusual MOSFIRE Observation

First, acquire a target using an offset star and a MOS mask alignment process with a MOSFIRE long slit.

Second, after offsetting to target, image the target in J-band through the alignment mask.  Use 9 coadds of 10 seconds each.

Finally, take Y and J longslit spectra of the target using an ABBA pattern with 3 repeats and standard MOSFIRE exposure parameters (e.g. 180s and 120s exposure times).

### 6) An Unusual MOSFIRE Observation: Alternate Observing Strategy

First, acquire a target using an offset star and a MOS mask alignment process with a MOSFIRE long slit.

Second, after offsetting to target, image the target in J-band through the alignment mask.  Use 9 coadds of 10 seconds each.

Third, instead of changing to the “science” long slit mask, take the Y and J spectra by offsetting the target away from the central alignment box.